In [1]:
%%time
import abem as ab
import numpy as np
from scipy.special import hankel1
from numpy.linalg import norm


# Test Problem 1
# Dirichlet boundary condition with phi chose to be the same
# as a point-source at the center of the square.
#
solver = ab.ExteriorHelmholtzSolver2D(ab.square())

frequency = 400.0 # frequency [Hz]
k = ab.frequency_to_wavenumber(frequency)

center_square = np.array([0.05, 0.05], dtype=np.float32)
boundary_condition = solver.dirichlet_boundary_condition()
centers = solver.geometry.centers()
for i in range(centers.shape[0]):
    r = norm(centers[i, :] - center_square)
    boundary_condition.f[i] = 1j * hankel1(0, k*r) / 4.0

boundary_incidence = ab.BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

exterior_points = np.array([[0.0000,  0.1500],
                           [0.0500,  0.1500],
                           [0.1000,  0.1500],
                           [0.0500, -0.1000]], dtype=np.float32)

exterior_incident_phi = np.zeros(exterior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 1")
print("==============\n")
print(boundary_solution)
print(sample_solution)

# Test Problem 2
# Neumann boundary condition with v chosen to be the same
# as a point-source at the center of the square.
#
boundary_condition = solver.neumann_boundary_condition()

center_square = np.array([0.05, 0.05], dtype=np.float32)
for i in range(centers.shape[0]):
    x = centers[i, :]
    r = x - center_square
    R = norm(r)
    if x[0] <= 1e-7:
        drbdn = -r[0]/R
    elif x[0] >= 0.1 - 1e-7:
        drbdn = r[0]/R
    elif x[1] <= 1e-7:
        drbdn = -r[1]/R
    elif x[1] >= 0.1 - 1e-7:
        drbdn = r[1]/R
    else:
        assert False, "r = ({}, {}) not properly handled.".format(r[0], r[1])
    boundary_condition.f[i] = -1j * k * drbdn * hankel1(1, k*R) / 4.0

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)

# Test Problem 3
# Neumann boundary condition with v=0 on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundary_condition = solver.neumann_boundary_condition()

source_point = np.array([0.5, 0.25], dtype=np.float32)
print("source_point = ", source_point)
for i in range(centers.shape[0]):
    x = centers[i, :]
    r = x - source_point
    R = norm(r)
    boundary_incidence.phi[i] = 1j * hankel1(0, k*R) / 4.0
    if x[0] <= 1e-7:
        drbdn = -r[0]/R
    elif x[0] >= 0.1 - 1e-7:
        drbdn = r[0]/R
    elif x[1] <= 1e-7:
        drbdn = -r[1]/R
    elif x[1] >= 0.1 - 1e-7:
        drbdn = r[1]/R
    else:
        assert False, "r = ({}, {}) not properly handled.".format(r[0], r[1])
    boundary_incidence.v[i] = -1j * k * drbdn * hankel1(1, k*R) / 4.0

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)

for i in range(exterior_points.shape[0]):
    R = norm(exterior_points[i, :] - source_point)
    exterior_incident_phi[i] = 1j * hankel1(0, k*R) / 4.0
    
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 3")
print("==============\n")
print(boundary_solution)
print(sample_solution)



Test Problem 1

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 7.306029319763184 (400.0 Hz)

index   Potential               Pressure                 Velocity                 Intensity

    1   1.1657e-01+2.3549e-01  -7.1318e+02+3.5303e+02i  -1.8592e+00-2.2407e-01i   6.2340e+02
    2   1.3824e-01+2.3854e-01  -7.2240e+02+4.1865e+02i  -2.6194e+00-3.1011e-01i   8.8121e+02
    3   1.5594e-01+2.4058e-01  -7.2859e+02+4.7227e+02i  -3.1497e+00-3.2514e-01i   1.0706e+03
    4   1.6620e-01+2.4160e-01  -7.3169e+02+5.0335e+02i  -3.4998e+00-3.2932e-01i   1.1975e+03
    5   1.6620e-01+2.4160e-01  -7.3169e+02+5.0335e+02i  -3.4998e+00-3.2933e-01i   1.1975e+03
    6   1.5594e-01+2.4058e-01  -7.2859e+02+4.7227e+02i  -3.1498e+00-3.2532e-01i   1.0706e+03
    7   1.3824e-01+2.3854e-01  -7.2240e+02+4.1865e+02i  -2.6194e+00-3.1024e-01i   8.8119e+02
    8   1.1657e-01+2.3549e-01  -7.1318e+02+3.5303e+02i  -1.8592e+00-2.2421e-01i   6.2338e+02
    9   1.1657e-01+2